In [ ]:
##### LOADING THE SN LIGHT CURVE #####

# SN TNS name
tnsname = ''

# SN discovery date
discovery_date = 0.0

# path to directory that contains SN and control light curves
source_dir = '/Users/sofiarest/Desktop/Supernovae/data/misc/2023ixf'

# filter of light curve to analyze
filt = 'o'

# MJD bin size in days of light curve to analyze
mjd_bin_size = 1.0

# define flags that define bad measurements (to filter out bad days in lc)
flags = 0x800000

##### LOADING CONTROL LIGHT CURVES #####

# number of control light curves to load
n_controls = 8

In [ ]:
import pandas as pd
import numpy as np
import sys,copy,random,math
from pdastro import pdastrostatsclass
from asym_gaussian import gauss2lc

# plotting
import matplotlib
import matplotlib.pyplot as plt

# suppress matplotlib warnings
import warnings
warnings.simplefilter('error', RuntimeWarning)
warnings.filterwarnings("ignore")

# plotting styles
plt.rc('axes', titlesize = 17)
plt.rc('xtick', labelsize = 14)
plt.rc('ytick', labelsize = 14)
plt.rc('legend', fontsize = 12)
plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=['red', 'orange', 'green', 'blue', 'purple', 'magenta'])

# for storing lc and control lcs 
global lcs
lcs = {}
# for storing info about the lc
global lc_info
# for storing detected tables
global detected
detected = None
# for storing efficiencies
global efficiencies
efficiencies = None

def AandB(A,B):
    return np.intersect1d(A,B,assume_unique=False)

def AnotB(A,B):
    return np.setdiff1d(A,B)

def AorB(A,B):
    return np.union1d(A,B)

def not_AandB(A,B):
    return np.setxor1d(A,B)

# get all indices of a dataframe
def get_ix(df):
    return df.index.values

# convert flux to magnitude 
def flux2mag(flux):
    return -2.5 * np.log10(flux) + 23.9

# convert magnitude to flux
def mag2flux(mag):
    return 10 ** ((mag - 23.9) / -2.5)

def apply_gaussian(lc, gaussian_sigma, sim_gauss=False, sim_peakmjd=None, sim_appmag=None, sim_sigma=None, print_=True, flag=0x800000):
    ix = get_ix(lc.t)
    if len(ix) < 1:
        raise RuntimeError('ERROR: not enough measurements to apply simulated gaussian')
    good_ix = AandB(ix, lc.ix_unmasked('Mask', flag)) # all good pre-SN indices

    # make sure there are no lingering simulations
    dropcols=[]
    for col in ['uJysim','SNRsim','simLC','SNRsimsum']:
        if col in lc.t.columns:
            dropcols.append(col)
    if len(dropcols) > 0:
        lc.t.drop(columns=dropcols,inplace=True)
    
    lc.t.loc[ix, 'SNR'] = 0.0
    lc.t.loc[good_ix,'SNR'] = lc.t.loc[good_ix,'uJy']/lc.t.loc[good_ix,'duJy']

    # add simulated gaussian
    if sim_gauss:
        if print_:
            print(f'# Adding simulated gaussian: peak at MJD {sim_peakmjd:0.2f}; apparent magnitude {sim_appmag:0.2f}; sigma- and sigma+ of {sim_sigma:0.2f} days')
        mjds = lc.t.loc[good_ix,'MJD']
        lc.t.loc[good_ix,'uJysim'] = lc.t.loc[good_ix,'uJy']
        lc.t.loc[ix,'simLC'] = 0.0

        # get simulated gaussian flux and add to light curve flux
        simflux = gauss2lc(mjds, sim_peakmjd, sim_sigma, sim_sigma, app_mag=sim_appmag)
        lc.t.loc[good_ix,'uJysim'] += simflux
        # get the simulated lc for all MJDs
        simflux_all = gauss2lc(lc.t.loc[ix,'MJDbin'], sim_peakmjd, sim_sigma, sim_sigma, app_mag=sim_appmag)
        lc.t.loc[ix,'simLC'] += simflux_all

        # make sure all bad rows have SNRsim = 0.0 so they have no impact on the rolling SNRsum
        lc.t.loc[ix,'SNRsim'] = 0.0
        # include simflux in the SNR
        lc.t.loc[good_ix,'SNRsim'] = lc.t.loc[good_ix,'uJysim']/lc.t.loc[good_ix,'duJy']

    new_gaussian_sigma = round(gaussian_sigma/lc_info['mjdbinsize'])
    windowsize = int(6 * new_gaussian_sigma)
    halfwindowsize = int(windowsize * 0.5) + 1
    if print_:
        print(f'# Sigma: {gaussian_sigma:0.2f} days; MJD bin size: {lc_info["mjdbinsize"]:0.2f} days; sigma: {new_gaussian_sigma:0.2f} bins; window size: {windowsize} bins')

    # calculate the rolling SNR sum
    dataindices = np.array(range(len(lc.t.loc[ix])) + np.full(len(lc.t.loc[ix]), halfwindowsize))
    temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='SNR', dtype=np.float64)
    temp[dataindices] = lc.t.loc[ix,'SNR']
    SNRsum = temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
    lc.t.loc[ix,'SNRsum'] = list(SNRsum[dataindices])
    # normalize it
    norm_temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='norm', dtype=np.float64)
    norm_temp[np.array(range(len(lc.t.loc[ix])) + np.full(len(lc.t.loc[ix]), halfwindowsize))] = np.ones(len(lc.t.loc[ix]))
    norm_temp_sum = norm_temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
    lc.t.loc[ix,'SNRsumnorm'] = list(SNRsum.loc[dataindices] / norm_temp_sum.loc[dataindices] * max(norm_temp_sum.loc[dataindices]))

    # calculate the rolling SNR sum for SNR with simflux
    if sim_gauss:
        temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='SNRsim', dtype=np.float64)
        temp[dataindices] = lc.t.loc[ix,'SNRsim']
        SNRsimsum = temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
        lc.t.loc[ix,'SNRsimsum'] = list(SNRsimsum.loc[dataindices])

    return lc

# load a single light curve
def load_lc(source_dir, control_index):
    if control_index == 0:
        filename = f'{source_dir}/{lc_info["tnsname"]}.{lc_info["filt"]}.{lc_info["mjdbinsize"]:0.2f}days.lc.txt'
        print(f'Loading light curve for SN {tnsname} at {filename} ...')
    else:
        filename = f'{source_dir}/controls/{lc_info["tnsname"]}_i{control_index:03d}.{lc_info["filt"]}.{lc_info["mjdbinsize"]:0.2f}days.lc.txt'
        print(f'Loading control light curve {control_index:03d} at {filename} ...')

    lcs[control_index] = pdastrostatsclass()
    try:
        lcs[control_index].load_spacesep(filename,delim_whitespace=True)
    except Exception as e:
        raise RuntimeError('ERROR: Could not load light curve at %s: %s' % (filename, str(e)))

# load control light curves
def load_control_lcs(source_dir, n_controls):
    for control_index in range(1,n_controls+1):
        load_lc(source_dir, control_index)

# load SN and control light curves
def load_all_lcs(source_dir, n_controls):
    load_lc(source_dir, 0)
    load_control_lcs(source_dir, n_controls)
    print('Success')

# set pre-SN (pre-discovery date) and post-SN (post-discovery date) indices
def set_discdate_ix():
    preSN_ix = lcs[0].ix_inrange(colnames=['MJDbin'], uplim=lc_info['discdate'], exclude_uplim=True)
    if len(preSN_ix) < 1:
        raise RuntimeError('ERROR: pre-SN indices must be 1 or greater')
    
    lc_info['preSN_ix'] = preSN_ix
    lc_info['postSN_ix'] = AnotB(get_ix(lcs[0].t), preSN_ix)
    
# fill lc_info
lc_info = {}
lc_info['tnsname'] = tnsname
lc_info['discdate'] = discovery_date
lc_info['filt'] = filt
lc_info['mjdbinsize'] = mjd_bin_size

# load SN and control lcs
load_all_lcs(source_dir, n_controls)

# set pre-SN and post-SN indices
set_discdate_ix()

# Cleaned original and averaged light curves

In [ ]:
color = 'red'

og_lc = pdastrostatsclass()
og_lc.load(filename=source_dir+'/2023ixf.o.lc.txt')

fig1, (ax1, ax2) = plt.subplots(2, constrained_layout=True)
fig1.set_figwidth(8)
fig1.set_figheight(7)
fig1.supxlabel('MJD')

# og lc 

ax1.set_ylim(-300,400)
ax1.set_xlim(og_lc.t.loc[0,'MJD'],discovery_date)
ax1.minorticks_on()
ax1.set_ylabel(r'Flux ($\mu$Jy)')
ax1.axhline(linewidth=1, color='k', zorder=0)

good_ix = og_lc.ix_unmasked('Mask',maskval=0x1|0x2|0x400000)
bad_ix = AnotB(og_lc.getindices(),good_ix)

# cleaned
ax1.errorbar(og_lc.t.loc[good_ix,'MJD'], og_lc.t.loc[good_ix,'uJy'], yerr=og_lc.t.loc[good_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax1.scatter(og_lc.t.loc[good_ix,'MJD'], og_lc.t.loc[good_ix,'uJy'], s=20, color=color, marker='o', alpha=0.5, label=f'Cleaned Measurements')

# flagged
ax1.errorbar(og_lc.t.loc[bad_ix,'MJD'], og_lc.t.loc[bad_ix,'uJy'], yerr=og_lc.t.loc[bad_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax1.scatter(og_lc.t.loc[bad_ix,'MJD'], og_lc.t.loc[bad_ix,'uJy'], s=20, facecolors='none', edgecolors=color, marker='o', alpha=0.5, label=f'Flagged Measurements')
ax1.legend(loc='upper right',fontsize=10)

# valid mjd ranges
valid_mjd_ranges = [[57463,57622], [57762,57983], [58120,58383], [58494,58741], [58822,59093], [59184,59445], [59566,59835], [59901,60085]]
for mjd_range in valid_mjd_ranges:
    ax1.axvline(mjd_range[0], color='magenta')
    ax1.axvline(mjd_range[1], color='magenta')

# averaged lc

ax2.set_ylim(-300,400)
ax2.set_xlim(og_lc.t.loc[0,'MJD'],discovery_date)
ax2.minorticks_on()
ax2.set_ylabel(r'Flux ($\mu$Jy)')
ax2.axhline(linewidth=1, color='k', zorder=0)

good_ix = lcs[0].ix_unmasked('Mask',maskval=flags)
bad_ix = AnotB(og_lc.getindices(),good_ix)

# cleaned
ax2.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax2.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=20, color=color, marker='o', alpha=0.5, label=f'Cleaned Measurements')

# flagged
ax2.errorbar(lcs[0].t.loc[bad_ix,'MJD'], lcs[0].t.loc[bad_ix,'uJy'], yerr=lcs[0].t.loc[bad_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax2.scatter(lcs[0].t.loc[bad_ix,'MJD'], lcs[0].t.loc[bad_ix,'uJy'], s=20, facecolors='none', edgecolors=color, marker='o', alpha=0.5, label=f'Flagged Measurements')
ax2.legend(loc='upper right',fontsize=10)

# Pre-SN and selected control light curve

In [ ]:
# select the control light curve to plot
select_index = 7

In [ ]:
color = 'red'
ctrl_color = 'blue'

fig1, (ax1, ax2) = plt.subplots(2, constrained_layout=True)
fig1.set_figwidth(8)
fig1.set_figheight(7)
fig1.supxlabel('MJD')

#ax1.set_ylim(-150,150)
ax1.set_xlim(lcs[0].t.loc[0,'MJD'],discovery_date)
ax1.minorticks_on()
ax1.get_xaxis().set_ticks([])
ax1.set_ylabel(r'Flux ($\mu$Jy)')
ax1.axhline(linewidth=1, color='k', zorder=0)

# pre-SN lc flux
good_ix = lcs[0].ix_unmasked('Mask',maskval=flags)
ax1.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5, zorder=20)
ax1.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=20, color=color, marker='o', alpha=0.5, label=f'Cleaned Pre-SN Lightcurve', zorder=20)
ax1.legend(loc='upper right',fontsize=10)

#ax2.set_ylim(-150,150)
ax2.set_xlim(lcs[0].t.loc[0,'MJD'],discovery_date)
ax2.minorticks_on()
ax2.set_ylabel(r'Flux ($\mu$Jy)')
ax2.axhline(linewidth=1, color='k', zorder=0)

# control lc flux
good_ix = lcs[select_index].ix_unmasked('Mask',maskval=flags)
ax2.errorbar(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], yerr=lcs[select_index].t.loc[good_ix,'duJy'], fmt='none', ecolor=ctrl_color, elinewidth=1, c=ctrl_color, alpha=0.5, zorder=10)
ax2.scatter(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], s=20, color=ctrl_color, marker='o', alpha=0.5, label=f'Cleaned Selected Control Lightcurve #{select_index}', zorder=10)
ax2.legend(loc='upper right',fontsize=10)

# Adding simulated gaussian bumps to a control light curve

In [ ]:
# select the control light curve to add the simulated bump to
select_index = 7

# sigmas of weighted gaussian rolling sums
gauss_sigmas = [13, 128]

# peak mjds of simulated bumps
sim_peak_mjds = [58600, 58600]

# gaussian sigmas of simulated bumps
sim_gauss_sigmas = [13, 128]

# peak fluxes of simulated bumps
sim_peaks = [30, 30]

# abs value of FOM y limits
ylimits = [40, 200]

In [ ]:
for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]
    sim_peak_mjd = sim_peak_mjds[i]
    sim_gauss_sigma = sim_gauss_sigmas[i]
    sim_peak = sim_peaks[i]

    # apply weighted gaussian rolling sum to control lc
    lcs[select_index] = apply_gaussian(lcs[select_index], gauss_sigma, print_=False)

    # apply weighted gaussian rolling sum to pre-SN lc
    lcs[0] = apply_gaussian(lcs[0], gauss_sigma, print_=False)

    # add simulated gaussian and apply gaussian weighted rolling sum
    simlc = copy.deepcopy(lcs[select_index])
    sim_appmag = -2.5 * np.log10(sim_peak) + 23.9
    simlc = apply_gaussian(simlc, gauss_sigma, sim_gauss=True, sim_peakmjd=sim_peak_mjd, sim_appmag=sim_appmag, sim_sigma=sim_gauss_sigma)

    ctrl_color = 'blue'

    fig2, (ax1, ax3) = plt.subplots(2, constrained_layout=True)
    fig2.set_figwidth(11)
    fig2.set_figheight(7)
    #fig2.supxlabel('MJD')

    # panel 1

    ax1.set_ylabel(r'Flux (µJy)')
    ax1.axhline(linewidth=1,color='k')
    ax1.tick_params(axis='y')
    ax1.get_xaxis().set_ticks([])
    ax1.minorticks_on()
    ax1.set_ylim(-50,50)
    ax1.set_xlim(sim_peak_mjd-sim_gauss_sigma*4, sim_peak_mjd+sim_gauss_sigma*4)

    # control lc flux
    good_ix = lcs[select_index].ix_unmasked('Mask',maskval=flags)
    ax1.errorbar(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], yerr=lcs[select_index].t.loc[good_ix,'duJy'], fmt='none', elinewidth=1, c=ctrl_color, alpha=1, zorder=20)
    ax1.scatter(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], s=20, color=ctrl_color, marker='o', alpha=1, zorder=20)

    # control lc FOM
    ax2 = ax1.twinx()
    ax2.minorticks_on()
    ax2.set_ylim(-ylimits[i],ylimits[i])
    ax2.set_ylabel('Figure of Merit (FOM)')
    ax2.scatter([0,1],[0,0], color=ctrl_color, alpha=1, marker='o', zorder=0, label=f'Cleaned Control Light Curve #{select_index}')
    ax2.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=ctrl_color, alpha=1, label=f'Weighted Gaussian Rolling Sum \nof Control Light Curve')
    plt.legend(loc='upper left',  bbox_to_anchor=(1.15,1), fontsize=10)
    ax2.tick_params(axis='y')
    #align_yaxis(ax1, ax2)

    # panel 2

    ax3.minorticks_on()
    ax3.set_xlabel('MJD')
    ax3.set_ylabel(r'Flux (µJy)')
    ax3.axhline(linewidth=1,color='k')
    ax3.set_ylim(-50,50)
    ax3.set_xlim(sim_peak_mjd-sim_gauss_sigma*4, sim_peak_mjd+sim_gauss_sigma*4)
    ax3.errorbar(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], yerr=simlc.t.loc[good_ix,'duJy'], fmt='none', elinewidth=1, c=ctrl_color, alpha=1, zorder=10)
    ax3.scatter(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], s=20, color=ctrl_color, marker='o', alpha=1, zorder=10)

    def my_gauss(x, sigma=sim_gauss_sigma, h=sim_peak, mid=sim_peak_mjd):
        variance = math.pow(sigma, 2)
        return h * math.exp(-math.pow(x-mid, 2)/(2*variance))
    x = np.linspace(sim_peak_mjd - 3*sim_gauss_sigma, sim_peak_mjd + 3*sim_gauss_sigma, 100)
    y = [my_gauss(xi) for xi in x]
    ax3.plot(x, y, color='black', linestyle='dashed')

    ax3.tick_params(axis='y')

    import matplotlib.ticker as ticker
    ax3.xaxis.set_major_locator(ticker.LinearLocator(6))
    ax3.xaxis.set_minor_locator(ticker.LinearLocator(31))

    ax4 = ax3.twinx()
    ax4.minorticks_on()
    ax4.set_ylim(-ylimits[i],ylimits[i])
    ax4.set_ylabel('Figure of Merit (FOM)')
    ax4.plot([0,1],[0,0], color='black', linestyle='dashed', alpha=1, label=f'Simulated Gaussian Bump')
    ax4.scatter([0,1],[0,0], color=ctrl_color, alpha=1, marker='o', label=f'Cleaned Control Light Curve #{select_index} \nwith Added Bump')
    ax4.plot(simlc.t['MJDbin'], simlc.t['SNRsimsum'], color=ctrl_color, alpha=1, label=f'Weighted Gaussian Rolling Sum of \nControl Light Curve with Added Bump')
    ax4.tick_params(axis='y')
    plt.legend(loc='upper left',  bbox_to_anchor=(1.15,1), fontsize=10)
    #align_yaxis(ax3, ax4)

    fig2.tight_layout()

# Weighted gaussian rolling sum of 3 illustratory simulated bumps

In [ ]:
# select the control light curve to add simulated bumps to
select_index = 7

# FOM detection limit
fom_limit = 11

# gaussian sigma of rolling gaussian weighted sum
gauss_sigma = 13

# number of simulated bumps to add
n_sim_gauss = 3 

# peak mjd for each simulated bump
sim_peak_mjds = [57900, 58600, 59250]

# gaussian sigma of each simulated bump
sim_gauss_sigmas = [13, 13, 13]

# peak flux of each simulated bump
sim_peaks = [10, 30, 60] 

In [ ]:
# apply weighted gaussian rolling sum to control lc
lcs[select_index] = apply_gaussian(lcs[select_index], gauss_sigma, print_=False)

ctrl_color = 'blue'

# plot 2
fig4, ax1 = plt.subplots(1, constrained_layout=True)
fig4.set_figwidth(8)
fig4.set_figheight(4)

ax1.set_xlim(57500,60090)
ax1.minorticks_on()
ax1.set_ylabel('Figure of Merit (FOM)')
ax1.set_xlabel('MJD')
ax1.axhline(linewidth=1, color='k', zorder=0)
ax1.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=ctrl_color, alpha=1, zorder=10)

# add simulated gausssians to selected control light curve
y = [7,20,30]
for i in range(n_sim_gauss): # for each simulated gaussian
    # add that simulated gaussian and apply gaussian weighted rolling sum
    simlc = copy.deepcopy(lcs[select_index])
    sim_appmag = -2.5 * np.log10(sim_peaks[i]) + 23.9
    simlc = apply_gaussian(simlc, gauss_sigma, sim_gauss=True, sim_peakmjd=sim_peak_mjds[i], sim_appmag=sim_appmag, sim_sigma=sim_gauss_sigmas[i])
    
    # plot it
    ax1.plot(simlc.t['MJDbin'], simlc.t['SNRsimsum'], linestyle='dashed', color=ctrl_color, zorder=0)
    ax1.text(sim_peak_mjds[i]+40, y[i], r'$m_{peak}$ = '+f'{sim_appmag:0.2f}', fontsize=11) #y=sim_peaks[i]

ax1.axhline(fom_limit, linewidth=1, color='black', linestyle='dashed') 
ax1.text(59550, fom_limit+1, r'$FOM_{limit}$ = '+str(fom_limit), fontsize=14)

ax1.plot([0,1],[0,0], color=ctrl_color, alpha=1, label='Weighted Gaussian Rolling Sum \nof Control Light Curve #7')
ax1.plot([0,1],[0,0], color=ctrl_color, linestyle='dashed', alpha=1, label=f'Weighted Gaussian Rolling Sum \nof Control Light Curve with\n{n_sim_gauss} Added Bumps')
plt.legend(loc='upper left',fontsize=10)

# Weighted gaussian rolling sum of pre-SN and control light curves

In [ ]:
# select the control light curve to highlight
select_index = 7

# skip certain control lcs
skip_ctr = [3]

# optionally remove a bad MJD range
mjd_range_remove = [57762,57983]

# sigmas of weighted gaussian rolling sums
gauss_sigmas = [5, 80, 200]

# plot illustratory histograms
plot = True

In [ ]:
# get FOM limits automatically for each gauss_sigma by getting the 3-sigma cut average of all control lc FOM

fom_limits = [None] * len(gauss_sigmas)

for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]
    
    allfom = pd.Series()
    c = 0
    for control_index in range(1, n_controls+1):
        if control_index in skip_ctr:
            continue
        
        ix = get_ix(lcs[control_index].t)
        if not(mjd_range_remove is None):
            mjd_ix_remove = lcs[control_index].ix_inrange(colnames='MJD',lowlim=mjd_range_remove[0], uplim=mjd_range_remove[1])
            ix = AnotB(ix, mjd_ix_remove)
        
        lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)
        allfom = pd.concat([allfom, lcs[control_index].t.loc[ix,'SNRsumnorm']], ignore_index=True)
        c += len(lcs[control_index].t)

    if plot:
        plt.figure(figsize=(4,2))
        plt.title(r'FOM distribution for ' + "\N{greek small letter tau}" + r"$_{G}$ = " + f'{gauss_sigma} days', fontsize=15)
        plt.hist(allfom, bins=10)
        plt.xlabel('FOM')

    df = pdastrostatsclass(columns=['SNRsumnorm'])
    df.t['SNRsumnorm'] = allfom

    df.calcaverage_sigmacutloop('SNRsumnorm', Nsigma=3.0, median_firstiteration=True)

    fom_limit1 = 3*df.statparams["stdev"]
    fom_limit2 = 5*df.statparams["stdev"]
    print(f'T_G={gauss_sigma}: stdev={df.statparams["stdev"]:0.2f}, 3*stdev={fom_limit1:0.2f}, 5*stdev={fom_limit2:0.2f}')

    fom_limits[i] = [round(fom_limit1), round(fom_limit2)]

print(f'\nFOM limits: {fom_limits}')

In [ ]:
for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]

    # apply rolling gaussian weighted sum to pre-SN and control light curves
    for control_index in range(0, n_controls+1):
        if control_index in skip_ctr:
            continue
        lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)

    color = 'red'
    ctrl_color = 'blue'
    else_color = 'green'

    fig, ax1 = plt.subplots(1, constrained_layout=True)
    fig.set_figwidth(8)
    fig.set_figheight(4)
    fig.supxlabel('MJD')

    ax1.set_xlim(57500,lc_info['discdate'])
    #ax1.set_ylim(-15, 25)
    ax1.minorticks_on()
    ax1.set_ylabel('Figure of Merit (FOM)')
    ax1.axhline(linewidth=1, color='k', zorder=0)

    # control lcs flux
    for control_index in range(1, n_controls):
        if control_index in skip_ctr or control_index == select_index:
            continue
        ax1.plot(lcs[control_index].t['MJDbin'], lcs[control_index].t['SNRsumnorm'], color=else_color, alpha=0.3, zorder=10)
    ax1.plot([0,1],[0,0], color=else_color, alpha=0.6, zorder=0, label=f'{n_controls-len(skip_ctr)-1} Control Light Curves')

    # selected control lc flux
    ax1.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=ctrl_color, alpha=0.7, zorder=20, label=f'Selected Control Lightcurve #{select_index}')

    # pre-SN lc flux
    ax1.plot(lcs[0].t['MJDbin'], lcs[0].t['SNRsumnorm'], color=color, label='Pre-SN Light Curve', zorder=40)

    for fom_limit in fom_limits[i]:
        ax1.axhline(fom_limit, linewidth=1, color='black', linestyle='dashed') 
        ax1.text(57600, fom_limit+0.5, r'$FOM_{limit}$ = '+str(fom_limit), fontsize=12)

    ax1.legend(loc='upper right',fontsize=10)

# Efficiencies

In [ ]:
# path to directory that contains bump analysis tables
table_dir = source_dir + '/bump_analysis/tables_test'

# sigmas of gaussian weighted rolling sum
gauss_sigmas = [5, 80, 200]

# simulated gaussian peak fluxes
peaks =  [5.75, 7.7, 10.3, 13.77, 18.42, 24.64, 32.95, 44.08, 58.96, 78.86, 105.48, 141.08, 188.71, 252.41, 337.61, 451.58, 604.02, 807.92, 1080.65, 1445.44]

In [ ]:
# load detected tables
def load_detected(table_dir, detected, gauss_sigmas, peaks):
    print(f'Loading detected tables in directory {table_dir} ...')
    detected = {}
    for gauss_sigma in gauss_sigmas:
        for peak in peaks:
            filename = f'{table_dir}/detected_{gauss_sigma}_{peak:0.2f}.txt'
            #print(f'Loading detected table with gauss_sigma {gauss_sigma} and peak flux {peak} at {filename} ...')
            try:
                detected[f'{gauss_sigma}_{peak:0.2f}'] = pdastrostatsclass()
                detected[f'{gauss_sigma}_{peak:0.2f}'].load_spacesep(filename=filename, delim_whitespace=True)
            except Exception as e:
                raise RuntimeError(f'ERROR: Could not load detected table with gauss_sigma {gauss_sigma} and peak flux {peak} at %s: %s' % (filename, str(e)))
    return detected

# load efficiencies
def load_efficiencies(table_dir, efficiencies):
    filename = f'{table_dir}/efficiencies.txt'
    print(f'Loading efficiencies at {filename} ...')
    try:
        efficiencies = pdastrostatsclass()
        efficiencies.load_spacesep(filename=filename, delim_whitespace=True)
    except Exception as e:
        raise RuntimeError('ERROR: Could not load efficiencies at %s: %s' % (filename, str(e)))
    return efficiencies

# load detected tables and efficiencies
def load_tables(table_dir, detected, efficiencies, gauss_sigmas, peaks):
    print()
    detected = load_detected(table_dir, detected, gauss_sigmas, peaks)
    efficiencies = load_efficiencies(table_dir, efficiencies)
    print('Success')
    return detected, efficiencies

# load detected tables and efficiencies
detected, efficiencies = load_tables(table_dir, detected, efficiencies, gauss_sigmas, peaks)

### Efficiencies using simulated Gaussians

In [ ]:
# select possible simulated gaussian sigmas
sim_gauss_sigmas = [[2, 5, 13], [30, 80, 120], [150, 200, 250, 300]]

In [ ]:
def get_gauss_coltitle(fom_limit, sim_gauss_sigma):
    return f'pct_detec_{fom_limit}_simsigma{sim_gauss_sigma:0.0f}'

for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]

    for fom_limit in fom_limits[i]:
        # recalculate efficiency for each width range
        gauss_sigma_ix = efficiencies.ix_equal(colnames=['gauss_sigma'], val=gauss_sigma)
        #print(efficiencies.t.loc[gauss_sigma_ix,'gauss_sigma'])
        for sim_gauss_sigma in sim_gauss_sigmas[i]:
            # recalculate efficiency
            for j in gauss_sigma_ix:
                peak = efficiencies.t.loc[j,'peak_flux']
                gauss_sigma = efficiencies.t.loc[j,'gauss_sigma']
                
                detected_ix = detected[f'{gauss_sigma}_{peak:0.2f}'].ix_inrange(colnames=['max_SNRsimsum'], lowlim=fom_limit)
                sigma_ix = detected[f'{gauss_sigma}_{peak:0.2f}'].ix_equal(colnames=['sim_gauss_sigma'], val=sim_gauss_sigma)
                percent_detected = len(AandB(detected_ix, sigma_ix))/len(sigma_ix) * 100
                efficiencies.t.loc[j, get_gauss_coltitle(fom_limit, sim_gauss_sigma)] = percent_detected

        # plot efficiencies

        fig1, ax1 = plt.subplots(1, constrained_layout=True)
        fig1.set_figwidth(6.5)
        fig1.set_figheight(3.5)

        # app mag
        ax1.set_title(r'Efficiencies for ' +  "\N{greek small letter tau}" + r"$_{G}$ = " + f'{gauss_sigma} days' + ', $FOM_{limit}$ = '+str(fom_limit), pad=10)
        ax1.axhline(linewidth=1,color='k')
        ax1.set_ylabel('Efficiency (%)', fontsize=14)
        for sim_gauss_sigma in sim_gauss_sigmas[i]:
            label = "\N{greek small letter tau}" + r"$'_{G}$ = " + f'{sim_gauss_sigma} days'
            ax1.plot(efficiencies.t.loc[gauss_sigma_ix, 'peak_appmag'], 
                     efficiencies.t.loc[gauss_sigma_ix, get_gauss_coltitle(fom_limit, sim_gauss_sigma)], label=label) #marker='o', 
        ax1.legend(loc='upper left', bbox_to_anchor=(1,1), fontsize=10)
        ax1.set_xlabel(r'$m_{peak}$ (app mag)')
        ax1.set_xlim(16,22)

        # abs mag
        ax3 = ax1.twiny()
        ax3.set_xticks(ax1.get_xticks() )
        ax3.set_xbound(ax1.get_xbound())
        ax3.set_xticklabels([round(x-29.04,2) for x in ax1.get_xticks()])
        ax3.set_xlabel("$m_{peak}$ (abs mag)")

efficiencies.write(f'efficiencies.txt')

### Efficiencies using a simulated eruption from a light curve

In [ ]:
# select possible simulated eruption days
sim_erup_days = [[70], [70], [70]]

In [ ]:
def get_erup_coltitle(fom_limit, sim_erup_day):
    return f'pct_detec_{fom_limit}_simerup{sim_erup_day:0.0f}'

for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]

    for fom_limit in fom_limits[i]:
        # recalculate efficiency for each width range
        gauss_sigma_ix = efficiencies.ix_equal(colnames=['gauss_sigma'], val=gauss_sigma)
        #print(efficiencies.t.loc[gauss_sigma_ix,'gauss_sigma'])
        for sim_erup_day in sim_erup_days[i]:
            # recalculate efficiency
            for j in gauss_sigma_ix:
                peak = efficiencies.t.loc[j,'peak_flux']
                gauss_sigma = efficiencies.t.loc[j,'gauss_sigma']
                
                detected_ix = detected[f'{gauss_sigma}_{peak:0.2f}'].ix_inrange(colnames=['max_SNRsimsum'], lowlim=fom_limit)
                erup_ix = detected[f'{gauss_sigma}_{peak:0.2f}'].ix_equal(colnames=['erup_days'], val=sim_erup_day)
                percent_detected = len(AandB(detected_ix, erup_ix))/len(erup_ix) * 100
                efficiencies.t.loc[j, get_erup_coltitle(fom_limit, sim_erup_day)] = percent_detected

        # plot efficiencies

        fig1, ax1 = plt.subplots(1, constrained_layout=True)
        fig1.set_figwidth(6.5)
        fig1.set_figheight(3.5)

        # app mag
        ax1.set_title(r'Efficiencies for ' +  "\N{greek small letter tau}" + r"$_{G}$ = " + f'{gauss_sigma} days' + ', $FOM_{limit}$ = '+str(fom_limit), pad=10)
        ax1.axhline(linewidth=1,color='k')
        ax1.set_ylabel('Efficiency (%)', fontsize=14)
        for sim_erup_day in sim_erup_days[i]:
            label = "\N{greek small letter tau}" + r"$'_{G}$ = " + f'{sim_erup_day} days'
            ax1.plot(efficiencies.t.loc[gauss_sigma_ix, 'peak_appmag'], 
                     efficiencies.t.loc[gauss_sigma_ix, get_erup_coltitle(fom_limit, sim_erup_day)], label=label) #marker='o', 
        ax1.legend(loc='upper left', bbox_to_anchor=(1,1), fontsize=10)
        ax1.set_xlabel(r'$m_{peak}$ (app mag)')
        ax1.set_xlim(16,22)

        # abs mag
        ax3 = ax1.twiny()
        ax3.set_xticks(ax1.get_xticks() )
        ax3.set_xbound(ax1.get_xbound())
        ax3.set_xticklabels([round(x-29.04,2) for x in ax1.get_xticks()])
        ax3.set_xlabel("$m_{peak}$ (abs mag)")

efficiencies.write(f'efficiencies2.txt')

# Contamination

In [ ]:
# skip certain control lcs
skip_ctr = [3]

# optionally remove a bad MJD range
mjd_range_remove = [57762,57983]

# sigmas of gaussian weighted rolling sum
gauss_sigmas = [5, 80, 200]

In [ ]:
def is_valid_mjd(mjd, mjd_range_remove):
    if mjd_range_remove is None: # all mjds valid
        return True
    elif mjd >= mjd_range_remove[0] and mjd <= mjd_range_remove[1]: # mjd must be outside mjd_range_remove
        return True
    else:
        return False

n_foms = 0
for l in fom_limits:
    n_foms += len(l)

stats = pdastrostatsclass()
stats.t['gauss_sigma'] = [np.nan] * n_foms
stats.t['fom'] = np.nan
stats.t['n_controls_triggered'] = 0
stats.t['percent_controls_triggered'] = np.nan
for control_index in range(1,n_controls+1):
    if not(control_index in skip_ctr):
        stats.t[f'n_detec_{control_index:02d}'] = 0

stats_index = 0
# for each gaussian sigma
for i in range(len(gauss_sigmas)):
    gauss_sigma = gauss_sigmas[i]
    for j in range(len(fom_limits[i])):
        fom_limit = fom_limits[i][j]
        stats.t.loc[stats_index, 'gauss_sigma'] = gauss_sigma
        stats.t.loc[stats_index, 'fom'] = fom_limit

        # for each control lc
        for control_index in range(1,n_controls+1):
            if control_index in skip_ctr:
                continue

            lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)

            """
            plt.figure()
            plt.plot(lcs[control_index].t['MJDbin'], lcs[control_index].t['SNRsumnorm'])
            plt.axhline(fom)
            """

            above_lim = False
            for k in range(len(lcs[control_index].t)):
                if lcs[control_index].t.loc[k, 'SNRsumnorm'] > fom_limit:
                    if not(above_lim) and is_valid_mjd(lcs[control_index].t.loc[k, 'MJD'], mjd_range_remove):
                        stats.t.loc[stats_index, f'n_detec_{control_index:02d}'] += 1
                    above_lim = True
                else:
                    above_lim = False
        
        for control_index in range(1,n_controls+1):
            if control_index in skip_ctr:
                continue
            if not(stats.t.loc[stats_index, f'n_detec_{control_index:02d}'] == 0):
                stats.t.loc[stats_index, 'n_controls_triggered'] += 1
        
        stats.t.loc[stats_index,'percent_controls_triggered'] = 100*stats.t.loc[stats_index,'n_controls_triggered']/(n_controls-len(skip_ctr))

        stats_index += 1

stats.write() # print
stats.write('contamination.txt') # save

# Zooming in on expected bump peak MJDs

In [ ]:
# expected peak mjds
bump_mjds = [57843.9334, 58781.3131]

# select fom limit to plot
fom_limit = 25

# select the gaussian sigma of weighted gaussian rolling sum
gauss_sigma = 100

# plot x limits scale
scale = 200

In [ ]:
lcs[0] = apply_gaussian(lcs[0], gauss_sigma, print_=False)
good_ix = lcs[0].ix_unmasked('Mask',maskval=flags)

In [ ]:
fig1, (ax1, ax2) = plt.subplots(2, constrained_layout=True)
fig1.set_figwidth(8)
fig1.set_figheight(7)

# panel 1

bump_mjd = bump_mjds[0]
ax1.set_xlim(bump_mjd-scale, bump_mjd+scale)
ax1.set_ylim(-100,100)
ax1.set_ylabel(r'Flux (µJy)')
ax1.axhline(linewidth=1,color='k')
#ax1.axvline(bump_mjd, label='Pre-SN Explosion', color='black')
ax1.tick_params(axis='y')
ax1.minorticks_on()

# flux
ax1.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax1.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=20, color=color, marker='o', alpha=1, label=f'Pre-SN Light Curve')

# FOM
ax1b = ax1.twinx()
ax1b.minorticks_on()
ax1b.set_ylim(-40,40)
ax1b.set_ylabel('Figure of Merit (FOM)')
ax1b.scatter([0,1],[0,0], color=color, alpha=1, marker='o', label=f'Pre-SN Light Curve')
ax1b.plot(lcs[0].t['MJDbin'], lcs[0].t['SNRsumnorm'], color=color, label='Weighted Gaussian Rolling Sum \nof Pre-SN Light Curve')
ax1b.axhline(fom_limit, linewidth=1, color='black', linestyle='dashed')
ax1b.text(57660, fom_limit+1, r'$FOM_{limit}$ = '+str(fom_limit), fontsize=14)

# panel 2

bump_mjd = bump_mjds[1]
ax2.set_xlabel('MJD')
ax2.set_xlim(bump_mjd-scale, bump_mjd+scale)
ax2.set_ylim(-75,75)
ax2.set_ylabel(r'Flux (µJy)')
ax2.axhline(linewidth=1,color='k')
#ax2.axvline(bump_mjd, label='Pre-SN Explosion',  color='black')
ax2.tick_params(axis='y')
ax2.minorticks_on()

# flux
ax2.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=color, elinewidth=1, c=color, alpha=0.5)
ax2.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=20, color=color, marker='o', alpha=1, label=f'Pre-SN Light Curve')

# FOM
ax2b = ax2.twinx()
ax2b.minorticks_on()
ax2b.set_ylim(-40,40)
ax2b.set_ylabel('Figure of Merit (FOM)')
ax2b.scatter([0,1],[0,0], color=color, alpha=1, marker='o', label=f'Pre-SN Light Curve')
ax2b.plot(lcs[0].t['MJDbin'], lcs[0].t['SNRsumnorm'], color=color, label='Weighted Gaussian Rolling Sum \nof Pre-SN Light Curve')
ax2b.axhline(fom_limit, linewidth=1, color='black', linestyle='dashed')
ax2b.text(58600, fom_limit+1, r'$FOM_{limit}$ = '+str(fom_limit), fontsize=14)

#plt.legend(loc='upper left',  bbox_to_anchor=(1.15,1), fontsize=10)